In [1]:
from pymongo import MongoClient
client = MongoClient()

import requests as req

import re
from datetime import datetime

In [2]:
def fetch(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
    }
    res = req.get(url, headers = headers)
    return res.text

In [3]:
html = fetch('http://www.smzdm.com/fenlei/')

In [4]:
reFenlei = re.compile('<h2[\s\S]*?\/fenlei\/(\w+)\/">[\s\S]*?<\/i>([\s\S]*?)<\/a>[\s\S]*?<\/h2>')

In [5]:
categories = reFenlei.findall(html)
categories

[('diannaoshuma', '电脑数码'),
 ('jiayongdianqi', '家用电器'),
 ('yundonghuwai', '运动户外'),
 ('fushixiebao', '服饰鞋包'),
 ('gehuhuazhuang', '个护化妆'),
 ('muyingyongpin', '母婴用品'),
 ('riyongbaihuo', '日用百货'),
 ('shipinbaojian', '食品保健'),
 ('lipinzhongbiao', '礼品钟表'),
 ('tushuyinxiang', '图书音像'),
 ('wanmoyueqi', '玩模乐器'),
 ('bangongshebei', '办公设备'),
 ('jiajujiazhuang', '家居家装'),
 ('qicheyongpin', '汽车消费'),
 ('jinronglvyou', '金融服务'),
 ('lvyouchuxing', '旅游出行'),
 ('wenhuayule', '文化娱乐'),
 ('fangchanzhiye', '房产置业')]

In [6]:
def get_first(l, default = None):
    if not l:
        return default
    else:
        return l[0]
def processItem(item, cate):
    record = {'fenlei': cate[0], 'new': True, 'fenlei_face': cate[1]}
    reStore = re.compile(r'class="mall" target="_blank">([\s\S]*?)<')
    reSourcer = re.compile(r'>爆料人：([\s\S]*?)<')
    reComments = re.compile(r'<i class="z-icon-comment"><\/i>(\d+)\s+<\/a>')
    reFavorites = re.compile(r'<i class="z-icon-star-empty"><\/i>\s+<span>(\d+)<\/span>')
    
    store = reStore.findall(item)
    sourcer = reSourcer.findall(item)
    comments = reComments.findall(item)
    favorites = reFavorites.findall(item)
    
    record['store'] = get_first(store)
    record['sourcer'] = get_first(sourcer)
    record['comments'] = int(comments[0])
    record['favs'] = int(favorites[0])
    
    rePid = re.compile(r'http:\/\/www\.smzdm\.com\/p\/(\d+)\/')
    record['pid'] = rePid.findall(item)[0]
    reTitle = re.compile(r'<h5.*?>\s*<a.*?>\s*<span.*?>(?P<tag>[\s\S]*?)<\/span>(?P<title>[\s\S]*?)<span.*?>(?P<price>[\s\S]*?)<\/span>\s*<\/a>')
    reUpvote = re.compile(r'<span class="unvoted-wrap">[\s\S]*?<span>(\d+)<\/span>')
    reTags = re.compile(r'<span class="feed-block-tags">([\s\S]*?)</span>')
    m = reTitle.search(item)
    record['label'] = m.group("tag")
    record['title'] = m.group("title").strip()
    record['price'] = m.group("price")
    [u, d] = reUpvote.findall(item)
    record['upvote'] = int(u)
    record['downvote'] = int(d)
    reTagContent = re.compile(r'<a.*?>([\s\S]*?)<\/a>')
    
    tags = reTagContent.findall(reTags.findall(item)[0])
    record['tags'] = tags
    return record

In [ ]:
client['dhub'].authenticate("data", "data")
SMZDM = client['dhub']['smzdm']
reItem = re.compile(r'<li class="feed-row-wide">([\s\S]*?)<\/li>')


for cate in categories:
    
    ent = "http://www.smzdm.com/fenlei/{}".format(cate[0])
    print("Processing {}".format(ent))
    for page in range(1, 400):
        url = "{}/p{}".format(ent, page)
        try:
            html = fetch(url)
            for item in reItem.findall(html):
                doc = processItem(item, cate)
                pid = SMZDM.find_one({"pid": doc['pid']})
                if not pid:
                    SMZDM.insert_one(doc)
                else:
                    SMZDM.find_one_and_replace({"pid": doc['pid']}, doc)
        except Exception as e:
            print(e)
            print("Failed at {}".format(url))

Processing http://www.smzdm.com/fenlei/diannaoshuma
'NoneType' object has no attribute 'group'
Failed at http://www.smzdm.com/fenlei/diannaoshuma/p11
list index out of range
Failed at http://www.smzdm.com/fenlei/diannaoshuma/p196
'NoneType' object has no attribute 'group'
Failed at http://www.smzdm.com/fenlei/diannaoshuma/p280
'NoneType' object has no attribute 'group'
Failed at http://www.smzdm.com/fenlei/diannaoshuma/p378
Processing http://www.smzdm.com/fenlei/jiayongdianqi
'NoneType' object has no attribute 'group'
Failed at http://www.smzdm.com/fenlei/jiayongdianqi/p69
'NoneType' object has no attribute 'group'
Failed at http://www.smzdm.com/fenlei/jiayongdianqi/p251
'NoneType' object has no attribute 'group'
Failed at http://www.smzdm.com/fenlei/jiayongdianqi/p261
Processing http://www.smzdm.com/fenlei/yundonghuwai
Processing http://www.smzdm.com/fenlei/fushixiebao
'NoneType' object has no attribute 'group'
Failed at http://www.smzdm.com/fenlei/fushixiebao/p20
'NoneType' object ha